In [20]:
import optuna
from optuna.samplers import TPESampler
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error
#from joblib import dump

In [21]:
# CUDA
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE.type

'cuda'

In [22]:
# Fix random seed
seed = 42
torch.manual_seed(seed)
if DEVICE.type == 'cuda':
	torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
sampler = TPESampler(seed=seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Pre-processing input data

In [23]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [24]:
results_df = pd.read_csv('../results_new/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio_new/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [25]:
# Make the target dataset
target_df = results_df[['total_time', 'brand_raw', 'count', 'l2_cache_size', 'l3_cache_size', 'l2_cache_line_size', 'l2_cache_associativity', 'ghz_advertised_friendly', 'benchmark']].copy()
# Rename columns to *_target
target_df = target_df.rename(columns={
    'total_time': 'total_time_target',
    'brand_raw': 'brand_raw_target',
    'count': 'count_target',
    'l2_cache_size': 'l2_cache_size_target',
    'l3_cache_size': 'l3_cache_size_target',
    'l2_cache_line_size': 'l2_cache_line_size_target',
    'l2_cache_associativity': 'l2_cache_associativity_target',
    'ghz_advertised_friendly': 'ghz_advertised_friendly_target',
})

dataset_df = pd.merge(results_df, target_df, how='inner', on='benchmark')
dataset_df = dataset_df[dataset_df['brand_raw'] != dataset_df['brand_raw_target']]
dataset_df.head(2)

,total_time,total_cpu_usage,max_ram_usage,brand_raw,count,l2_cache_size,l3_cache_size,l2_cache_line_size,l2_cache_associativity,benchmark,ghz_actual_friendly,ghz_advertised_friendly,total_time_target,brand_raw_target,count_target,l2_cache_size_target,l3_cache_size_target,l2_cache_line_size_target,l2_cache_associativity_target,ghz_advertised_friendly_target
5,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,45.91,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496
6,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,25.77,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496


In [26]:
# remove one computer for testing
g_train = dataset_df[(dataset_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (dataset_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
g_test = dataset_df[dataset_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [27]:
mm_df = dataset_df[dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
mm_train = mm_df[(mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (mm_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
mm_test = mm_df[mm_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [28]:
st_df = dataset_df[~dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
st_train = st_df[(st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (st_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
st_test = st_df[st_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [29]:
# load test dataset
g_test = pd.read_csv('csv/g_test.csv')
st_test = pd.read_csv('csv/st_test.csv')
mm_test = pd.read_csv('csv/mm_test.csv')

In [30]:
target = 'total_time_target'
features = mm_test.columns.copy().drop(target).drop(['benchmark','brand_raw', 'brand_raw_target'])
features_st = features.copy().drop(['count', 'count_target'])

In [31]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
scaler_g = StandardScaler()
X_g_train = scaler_g.fit_transform(X_g_train)
X_g_test = scaler_g.transform(X_g_test)

## convert to tensor
X_g_train = torch.tensor(X_g_train, dtype=torch.float32).unsqueeze(1)
X_g_test = torch.tensor(X_g_test, dtype=torch.float32).unsqueeze(1)
y_g_train = torch.tensor(y_g_train.values, dtype=torch.float32).view(-1, 1)
y_g_test = torch.tensor(y_g_test.values, dtype=torch.float32).view(-1, 1)

In [32]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
scaler_st = StandardScaler()
X_st_train = scaler_st.fit_transform(X_st_train)
X_st_test = scaler_st.transform(X_st_test)

## convert to tensor
X_st_train = torch.tensor(X_st_train, dtype=torch.float32).unsqueeze(1)
X_st_test = torch.tensor(X_st_test, dtype=torch.float32).unsqueeze(1)
y_st_train = torch.tensor(y_st_train.values, dtype=torch.float32).view(-1, 1)
y_st_test = torch.tensor(y_st_test.values, dtype=torch.float32).view(-1, 1)

In [33]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
scaler_mm = StandardScaler()
X_mm_train = scaler_mm.fit_transform(X_mm_train)
X_mm_test = scaler_mm.transform(X_mm_test)

## convert to tensor
X_mm_train = torch.tensor(X_mm_train, dtype=torch.float32).unsqueeze(1)
X_mm_test = torch.tensor(X_mm_test, dtype=torch.float32).unsqueeze(1)
y_mm_train = torch.tensor(y_mm_train.values, dtype=torch.float32).view(-1, 1)
y_mm_test = torch.tensor(y_mm_test.values, dtype=torch.float32).view(-1, 1)

In [34]:
if DEVICE.type == 'cuda':
	# move to DEVICE
	X_g_train = X_g_train.to(DEVICE)
	y_g_train = y_g_train.to(DEVICE)
	X_g_test = X_g_test.to(DEVICE)
	y_g_test = y_g_test.to(DEVICE)

	X_st_train = X_st_train.to(DEVICE)
	y_st_train = y_st_train.to(DEVICE)
	X_st_test = X_st_test.to(DEVICE)
	y_st_test = y_st_test.to(DEVICE)

	X_mm_train = X_mm_train.to(DEVICE)
	y_mm_train = y_mm_train.to(DEVICE)
	X_mm_test = X_mm_test.to(DEVICE)
	y_mm_test = y_mm_test.to(DEVICE)

# Model

In [35]:
class TransformerModel(nn.Module):
	def __init__(self, input_dim_numeric, embedding_dim, seq_length, num_heads, num_layers, output_dim):
		super(TransformerModel, self).__init__()
		# layers
		self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads),
            num_layers=num_layers
        )
		self.fc_numeric = nn.Linear(input_dim_numeric, embedding_dim)
		self.fc_final = nn.Linear(embedding_dim * (seq_length + 1), output_dim)
	
	def forward(self, numeric_inputs, seq_inputs):
		numeric_embeddings = self.fc_numeric(numeric_inputs)
		seq_embeddings = self.transformer_encoder(seq_inputs)
		combined = torch.cat((seq_embeddings.flatten(1), numeric_embeddings), dim=1)
		output = self.fc_final(combined)
		return output

In [36]:
def objective(trial: optuna.Trial, X_train, X_emb_train, y_train, X_test, y_test, input_num_dim, input_emb_dim, output_dim):
	# Definimos los hiperparámetros a buscar
	num_heads = trial.suggest_int('num_heads', 1, 8)
	num_layers = trial.suggest_int('num_layers', 1, 6)
	learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
	weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
	num_epochs = trial.suggest_int('num_epochs', 10, 100)

	# model initialization 
	model = TransformerModel(input_num_dim, input_emb_dim, 5000, num_heads, num_layers, output_dim)
	if DEVICE.type == 'cuda':
		model = model.to(DEVICE)
	criterion = nn.MSELoss()
	optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
	# training
	model.train()
	for epoch in range(num_epochs):
		optimizer.zero_grad()
		output = model(X_train)
		loss = criterion(output, y_train)
		loss.backward()
		optimizer.step()
	# evaluation
	model.eval()
	with torch.no_grad():
		predictions = model(X_test)
		val_loss = criterion(predictions, y_test)

		# trial.report(val_loss.item(), epoch+1)
		# if trial.should_prune():
		# 	raise optuna.TrialPruned()
	print(f"Trial: {trial.number} - Loss: {loss.item()} - Val Loss: {val_loss.item()}")
	return val_loss.item()

# Hyperparameters Optimization

In [37]:
n_trials = 25
study_g = None
study_st = None
study_mm = None

## General

In [38]:
# configuration optuna
study_g = optuna.create_study(direction='minimize', sampler=sampler)
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test, len(features), 1), n_trials=n_trials)

[I 2024-07-03 07:45:51,581] A new study created in memory with name: no-name-504ff626-0f26-4741-9f15-d1fb61d4cf84
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
[I 2024-07-03 07:45:58,988] Trial 0 finished with value: 587.005615234375 and parameters: {'num_heads': 3, 'model_dim': 183, 'num_layers': 5, 'dropout': 0.3394633936788146, 'learning_rate': 2.9380279387035334e-05, 'weight_decay': 2.9375384576328295e-05, 'num_epochs': 15}. Best is trial 0 with value: 587.005615234375.


Trial: 0 - Loss: 573.3038940429688 - Val Loss: 587.005615234375


[I 2024-07-03 07:46:26,691] Trial 1 finished with value: 65.44300842285156 and parameters: {'num_heads': 7, 'model_dim': 280, 'num_layers': 5, 'dropout': 0.10823379771832098, 'learning_rate': 0.008123245085588688, 'weight_decay': 0.00314288089084011, 'num_epochs': 29}. Best is trial 1 with value: 65.44300842285156.


Trial: 1 - Loss: 267.2928161621094 - Val Loss: 65.44300842285156


[I 2024-07-03 07:46:33,623] Trial 2 finished with value: 616.79443359375 and parameters: {'num_heads': 2, 'model_dim': 30, 'num_layers': 2, 'dropout': 0.3099025726528951, 'learning_rate': 0.00019762189340280086, 'weight_decay': 7.476312062252303e-05, 'num_epochs': 65}. Best is trial 1 with value: 65.44300842285156.


...

Trial: 22 - Loss: 266.6722106933594 - Val Loss: 73.73278045654297


[I 2024-07-03 07:55:15,160] Trial 23 finished with value: 203.9698028564453 and parameters: {'num_heads': 4, 'model_dim': 172, 'num_layers': 5, 'dropout': 0.26166750169446507, 'learning_rate': 0.004567652233675725, 'weight_decay': 0.0007663713378426931, 'num_epochs': 11}. Best is trial 1 with value: 65.44300842285156.


Trial: 23 - Loss: 331.7026062011719 - Val Loss: 203.9698028564453


[I 2024-07-03 07:55:24,779] Trial 24 finished with value: 308.0043029785156 and parameters: {'num_heads': 3, 'model_dim': 135, 'num_layers': 4, 'dropout': 0.1818506671679665, 'learning_rate': 0.0016176302658366711, 'weight_decay': 1.0086280568678738e-05, 'num_epochs': 26}. Best is trial 1 with value: 65.44300842285156.


Trial: 24 - Loss: 383.8891906738281 - Val Loss: 308.0043029785156


In [39]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida de validación: {trial.value}')

Número de pruebas: 25
Mejor prueba: 1
Mejores parametros: {'num_heads': 7, 'model_dim': 280, 'num_layers': 5, 'dropout': 0.10823379771832098, 'learning_rate': 0.008123245085588688, 'weight_decay': 0.00314288089084011, 'num_epochs': 29}
Mejor valor de pérdida de validación: 65.44300842285156


## Single Thread

In [40]:
# configuration optuna
study_st = optuna.create_study(direction='minimize', sampler=sampler)
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test, len(features_st), 1), n_trials=n_trials)

[I 2024-07-03 07:55:24,799] A new study created in memory with name: no-name-dc29c454-4add-4623-b461-8651c59556d0
[I 2024-07-03 07:55:34,940] Trial 0 finished with value: 62.06943893432617 and parameters: {'num_heads': 7, 'model_dim': 112, 'num_layers': 1, 'dropout': 0.42618457138193366, 'learning_rate': 0.001319994226153501, 'weight_decay': 0.0015382308040279, 'num_epochs': 80}. Best is trial 0 with value: 62.06943893432617.


Trial: 0 - Loss: 307.4320068359375 - Val Loss: 62.06943893432617


[I 2024-07-03 07:55:35,386] Trial 1 finished with value: 507.5620422363281 and parameters: {'num_heads': 1, 'model_dim': 25, 'num_layers': 1, 'dropout': 0.4452413703502375, 'learning_rate': 0.0007411299781083245, 'weight_decay': 9.833181933644887e-05, 'num_epochs': 15}. Best is trial 0 with value: 62.06943893432617.


Trial: 1 - Loss: 744.2530517578125 - Val Loss: 507.5620422363281


[I 2024-07-03 07:55:41,791] Trial 2 finished with value: 220.41946411132812 and parameters: {'num_heads': 3, 'model_dim': 69, 'num_layers': 5, 'dropout': 0.35502298854208525, 'learning_rate': 0.0045881565491609705, 'weight_decay': 0.00026100256506134784, 'num_epochs': 20}. Best is trial 0 with value: 62.06943893432617.


...

Trial: 22 - Loss: 281.1385192871094 - Val Loss: 37.341793060302734


[I 2024-07-03 08:08:15,549] Trial 23 finished with value: 105.7842025756836 and parameters: {'num_heads': 6, 'model_dim': 234, 'num_layers': 5, 'dropout': 0.2691750909442026, 'learning_rate': 0.0003436782144940018, 'weight_decay': 0.007187818462626211, 'num_epochs': 98}. Best is trial 17 with value: 37.23849105834961.


Trial: 23 - Loss: 348.4277648925781 - Val Loss: 105.7842025756836


[I 2024-07-03 08:09:12,858] Trial 24 finished with value: 37.23106002807617 and parameters: {'num_heads': 4, 'model_dim': 212, 'num_layers': 6, 'dropout': 0.19657967657361075, 'learning_rate': 0.0015831434455046685, 'weight_decay': 0.0007568934817355099, 'num_epochs': 100}. Best is trial 24 with value: 37.23106002807617.


Trial: 24 - Loss: 281.19830322265625 - Val Loss: 37.23106002807617


In [41]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 25
Mejor prueba: 24
Mejores parametros: {'num_heads': 4, 'model_dim': 212, 'num_layers': 6, 'dropout': 0.19657967657361075, 'learning_rate': 0.0015831434455046685, 'weight_decay': 0.0007568934817355099, 'num_epochs': 100}
Mejor valor de pérdida en validación: 37.23106002807617


## Multi Thread

In [42]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize', sampler=sampler)
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test, len(features), 1), n_trials=n_trials)

[I 2024-07-03 08:09:12,875] A new study created in memory with name: no-name-6519a2ac-7050-4a61-bc66-b9c1f9dc1c17
[I 2024-07-03 08:09:20,085] Trial 0 finished with value: 828.8079223632812 and parameters: {'num_heads': 8, 'model_dim': 152, 'num_layers': 3, 'dropout': 0.22035132392670786, 'learning_rate': 7.153547794693153e-05, 'weight_decay': 1.2902113024567147e-05, 'num_epochs': 65}. Best is trial 0 with value: 828.8079223632812.


Trial: 0 - Loss: 133.3551025390625 - Val Loss: 828.8079223632812


[I 2024-07-03 08:09:22,447] Trial 1 finished with value: 1094.959716796875 and parameters: {'num_heads': 5, 'model_dim': 35, 'num_layers': 2, 'dropout': 0.46330635438666146, 'learning_rate': 5.232216089948759e-05, 'weight_decay': 2.7207248059486674e-05, 'num_epochs': 54}. Best is trial 0 with value: 828.8079223632812.


Trial: 1 - Loss: 200.1089630126953 - Val Loss: 1094.959716796875


[I 2024-07-03 08:09:30,323] Trial 2 finished with value: 882.26953125 and parameters: {'num_heads': 8, 'model_dim': 144, 'num_layers': 5, 'dropout': 0.4046478461314871, 'learning_rate': 5.163124910488014e-05, 'weight_decay': 0.0015298506868937445, 'num_epochs': 43}. Best is trial 0 with value: 828.8079223632812.


...

Trial: 22 - Loss: 107.12117004394531 - Val Loss: 506.086669921875


[I 2024-07-03 08:10:40,470] Trial 23 finished with value: 616.2301635742188 and parameters: {'num_heads': 7, 'model_dim': 357, 'num_layers': 4, 'dropout': 0.1506422952327002, 'learning_rate': 0.0016771596843081652, 'weight_decay': 5.3371988517039574e-05, 'num_epochs': 18}. Best is trial 9 with value: 389.2778625488281.


Trial: 23 - Loss: 59.77368927001953 - Val Loss: 616.2301635742188


[I 2024-07-03 08:10:51,772] Trial 24 finished with value: 528.7885131835938 and parameters: {'num_heads': 8, 'model_dim': 472, 'num_layers': 6, 'dropout': 0.19178868835085539, 'learning_rate': 0.0010973253638146487, 'weight_decay': 0.00018781161475919233, 'num_epochs': 36}. Best is trial 9 with value: 389.2778625488281.


Trial: 24 - Loss: 107.22895050048828 - Val Loss: 528.7885131835938


In [43]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 25
Mejor prueba: 9
Mejores parametros: {'num_heads': 8, 'model_dim': 320, 'num_layers': 1, 'dropout': 0.14058861714641285, 'learning_rate': 0.0009783749110062348, 'weight_decay': 1.0355826161899173e-05, 'num_epochs': 24}
Mejor valor de pérdida en validación: 389.2778625488281


# Training

In [44]:
models_folder = '../models/transformer'
output_dim = 1

In [45]:
#dump(scaler_g, f'{models_folder}/scaler_g.joblib')
#dump(scaler_st, f'{models_folder}/scaler_st.joblib')
#dump(scaler_mm, f'{models_folder}/scaler_mm.joblib')

## General

In [46]:
input_dim = len(features)
# hyperparameters
if study_g is not None:
	num_heads = study_g.best_trial.params['num_heads']
	model_dim = study_g.best_trial.params['model_dim']
	num_layers = study_g.best_trial.params['num_layers']
	dropout = study_g.best_trial.params['dropout']
	lr = study_g.best_trial.params['learning_rate']
	wd = study_g.best_trial.params['weight_decay']
	num_epochs = study_g.best_trial.params['num_epochs']
else:
	num_heads = 6
	model_dim = 294
	num_layers = 6
	dropout = 0.49925267200585316
	lr = 0.007097392833743549
	wd = 0.0017343770121671484
	num_epochs = 35

In [47]:
# general model initialization
model_g = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_g = model_g.to(DEVICE)
criterion_g = nn.MSELoss()
optimizer_g = optim.AdamW(model_g.parameters(), lr=lr, weight_decay=wd)

model_g.train()

for epoch in range(num_epochs):
	optimizer_g.zero_grad()
	output = model_g(X_g_train)
	loss = criterion_g(output, y_g_train)
	loss.backward()
	optimizer_g.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_g.eval()
		with torch.no_grad():
			val_predictions = model_g(X_g_test)
			val_loss = criterion_g(val_predictions, y_g_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_g.train()

Epoch 10/29, Loss: 270.0231628417969, Val Loss: 56.22865676879883
Epoch 20/29, Loss: 267.6117858886719, Val Loss: 93.45111846923828
Epoch 29/29, Loss: 267.2730407714844, Val Loss: 66.2825698852539


In [49]:
model_g.eval()
with torch.no_grad():
	preds = model_g(X_g_test).cpu().numpy().flatten()
mse = mean_squared_error(y_g_test.cpu().numpy().flatten(), preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mean_absolute_error(y_g_test.cpu().numpy().flatten(), preds)}")

MSE: 66.28256225585938 - RMSE: 8.141409873962402 - MAE: 5.510138511657715


In [50]:
# save model
torch.save(model_g, f'{models_folder}/general.pt')

## Single Thread

In [51]:
input_dim = len(features_st)
# hyperparameters
if study_st is not None:
	num_heads = study_st.best_trial.params['num_heads']
	model_dim = study_st.best_trial.params['model_dim']
	num_layers = study_st.best_trial.params['num_layers']
	dropout = study_st.best_trial.params['dropout']
	lr = study_st.best_trial.params['learning_rate']
	wd = study_st.best_trial.params['weight_decay']
	num_epochs = study_st.best_trial.params['num_epochs']
else:
	num_heads = 6
	model_dim = 192
	num_layers = 4
	dropout = 0.3731512093597947
	lr = 0.0027591245533166004
	wd = 0.0014100590768903643
	num_epochs = 78

In [52]:
# single thread model initialization
model_st = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_st = model_st.to(DEVICE)
criterion_st = nn.MSELoss()
optimizer_st = optim.AdamW(model_st.parameters(), lr=lr, weight_decay=wd)

model_st.train()

for epoch in range(num_epochs):
	optimizer_st.zero_grad()
	output = model_st(X_st_train)
	loss = criterion_st(output, y_st_train)
	loss.backward()
	optimizer_st.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_st.eval()
		with torch.no_grad():
			val_predictions = model_st(X_st_test)
			val_loss = criterion_st(val_predictions, y_st_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_st.train()

Epoch 10/100, Loss: 468.1701965332031, Val Loss: 215.08013916015625
Epoch 20/100, Loss: 387.3863220214844, Val Loss: 137.333984375
Epoch 30/100, Loss: 329.1274719238281, Val Loss: 81.00090026855469
Epoch 40/100, Loss: 295.8929138183594, Val Loss: 49.859031677246094
Epoch 50/100, Loss: 283.1757507324219, Val Loss: 38.61391067504883
Epoch 60/100, Loss: 281.1542053222656, Val Loss: 37.20664596557617
Epoch 70/100, Loss: 281.47821044921875, Val Loss: 37.50769805908203
Epoch 80/100, Loss: 281.2610778808594, Val Loss: 37.17526626586914
Epoch 90/100, Loss: 281.125244140625, Val Loss: 37.208595275878906
Epoch 100/100, Loss: 281.1525573730469, Val Loss: 37.19477844238281


In [53]:
model_st.eval()
with torch.no_grad():
	preds = model_st(X_st_test).cpu().numpy().flatten()
mse = mean_squared_error(y_st_test.cpu().numpy().flatten(), preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mean_absolute_error(y_st_test.cpu().numpy().flatten(), preds)}")

MSE: 37.19477844238281 - RMSE: 6.098752021789551 - MAE: 3.26633358001709


In [54]:
# save model
torch.save(model_st, f'{models_folder}/single_thread.pt')

## Multi Thread

In [55]:
input_dim = len(features)
# hyperparameters
if study_mm is not None:
	num_heads = study_mm.best_trial.params['num_heads']
	model_dim = study_mm.best_trial.params['model_dim']
	num_layers = study_mm.best_trial.params['num_layers']
	dropout = study_mm.best_trial.params['dropout']
	lr = study_mm.best_trial.params['learning_rate']
	wd = study_mm.best_trial.params['weight_decay']
	num_epochs = study_mm.best_trial.params['num_epochs']
else:
	num_heads = 7
	model_dim = 413
	num_layers = 1
	dropout = 0.2897232837483553
	lr = 0.0016919131094273875
	wd = 0.0008901136789000729
	num_epochs = 49

In [56]:
# multi thread model initialization
model_mm = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_mm = model_mm.to(DEVICE)
criterion_mm = nn.MSELoss()
optimizer_mm = optim.AdamW(model_mm.parameters(), lr=lr, weight_decay=wd)

model_mm.train()

for epoch in range(num_epochs):
	optimizer_mm.zero_grad()
	output = model_mm(X_mm_train)
	loss = criterion_mm(output, y_mm_train)
	loss.backward()
	optimizer_mm.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_mm.eval()
		with torch.no_grad():
			val_predictions = model_mm(X_mm_test)
			val_loss = criterion_mm(val_predictions, y_mm_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_mm.train()

Epoch 10/24, Loss: 87.99156188964844, Val Loss: 586.1416625976562
Epoch 20/24, Loss: 66.74063873291016, Val Loss: 452.10601806640625
Epoch 24/24, Loss: 61.3031120300293, Val Loss: 407.0425109863281


In [57]:
model_mm.eval()
with torch.no_grad():
	preds = model_mm(X_mm_test).cpu().numpy().flatten()
mse = mean_squared_error(y_mm_test.cpu().numpy().flatten(), preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mean_absolute_error(y_mm_test.cpu().numpy().flatten(), preds)}")

MSE: 407.0425109863281 - RMSE: 20.175294876098633 - MAE: 20.15708351135254


In [58]:
# save model
torch.save(model_mm, f'{models_folder}/multi_thread.pt')

# Load models

In [59]:
model_g = torch.load(f'{models_folder}/general.pt').to(DEVICE)
model_st = torch.load(f'{models_folder}/single_thread.pt').to(DEVICE)
model_mm = torch.load(f'{models_folder}/multi_thread.pt').to(DEVICE)

In [60]:
# 1 instance prediction
def predict(model, X):
	model.eval()
	with torch.no_grad():
		prediction = model(X)
	return prediction

def describe_val(model, X, y):
	min_instance = {"prediction": float('inf'), "actual": 0, "index": 0}
	max_instance = {"prediction": 0, "actual": 0, "index": 0}
	
	predictions = predict(model, X).cpu().numpy().flatten()
	index_min = np.argmin(np.abs(predictions - y.cpu().numpy().flatten()))
	min_instance["prediction"] = predictions[index_min]
	min_instance["actual"] = y.cpu().numpy().flatten()[index_min]
	min_instance["index"] = index_min
	index_max = np.argmax(np.abs(predictions - y.cpu().numpy().flatten()))
	max_instance["prediction"] = predictions[index_max]
	max_instance["actual"] = y.cpu().numpy().flatten()[index_max]
	max_instance["index"] = index_max

	return min_instance, max_instance, predictions

In [68]:
# general model
print("Validation set general model")
min_instance, max_instance, predictions = describe_val(model_g, X_g_test, y_g_test)
errors = np.abs(predictions - y_g_test.cpu().numpy().flatten())
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_g_test.cpu().numpy().flatten())} | Std actual: {np.std(y_g_test.cpu().numpy().flatten())}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("---")
print("Min instance")
print(g_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(g_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set general model
Mean prediction: 24.186809539794922 | Std actual: 2.045403562078718e-06
Mean actual: 27.556499481201172 | Std actual: 7.411326885223389
Mean Error: 5.510138511657715 | Std Error: 5.993408679962158
---
Min instance
total_time                                                       17.87
total_cpu_usage                                                   0.99
max_ram_usage                                                15.527344
brand_raw                         12th Gen Intel(R) Core(TM) i5-12400F
count                                                               12
l2_cache_size                                                      7.5
l3_cache_size                                                     18.0
l2_cache_line_size                                                1280
l2_cache_associativity                                               7
benchmark                                                     N_Queens
ghz_actual_friendly                            

In [69]:
# single thread model
print("Validation set single thread model")
min_instance, max_instance, predictions = describe_val(model_st, X_st_test, y_st_test)
errors = np.abs(predictions - y_st_test.cpu().numpy().flatten())
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_st_test.cpu().numpy().flatten())} | Std actual: {np.std(y_st_test.cpu().numpy().flatten())}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(st_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(st_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model
Mean prediction: 24.714994430541992 | Std actual: 4.625657311407849e-06
Mean actual: 24.564001083374023 | Std actual: 6.096883296966553
Mean Error: 3.26633358001709 | Std Error: 5.150325298309326
Min instance
total_time                                                             24.7
total_cpu_usage                                                        0.99
max_ram_usage                                                     10.207031
brand_raw                         Intel(R) Xeon(R) CPU E5-2623 v3 @ 3.00GHz
count                                                                     8
l2_cache_size                                                           2.0
l3_cache_size                                                          10.0
l2_cache_line_size                                                      256
l2_cache_associativity                                                    2
benchmark                                                          N_Que

In [70]:
# multi thread model
print("Validation set multi thread model")
min_instance, max_instance, predictions = describe_val(model_mm, X_mm_test, y_mm_test)
errors = np.abs(predictions - y_mm_test.cpu().numpy().flatten())
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_mm_test.cpu().numpy().flatten())} | Std actual: {np.std(y_mm_test.cpu().numpy().flatten())}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(mm_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(mm_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set multi thread model
Mean prediction: 16.376916885375977 | Std actual: 0.00497777434065938
Mean actual: 36.534000396728516 | Std actual: 0.8569851517677307
Mean Error: 20.15708351135254 | Std Error: 0.8569996356964111
Min instance
total_time                                                            37.51
total_cpu_usage                                                        6.15
max_ram_usage                                                   2350.140625
brand_raw                         Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz
count                                                                     8
l2_cache_size                                                           1.0
l3_cache_size                                                           8.0
l2_cache_line_size                                                      256
l2_cache_associativity                                                    6
benchmark                                                       MATRIX_M